In [1]:
%run helpers.py
%run classification.py
%run transformations.py
%run model.py
%run prediction.py

In [2]:
import argparse
import os
import torch
import classification
import helpers
import transformations as tr
import model
import prediction
from torchvision import transforms
from torch.utils.data import DataLoader

In [3]:
def parse_cmdline_arguments():
    """Parse command line arguments"""
    parser = argparse.ArgumentParser(description='Player and Team Recognition...')
    parser.add_argument('--perform', type=str.lower,
                        choices=['train', 'predict', 'predict_and_color', 'train_and_predict'],
                        default='train_and_predict', help='Perform training or prediction or both')
    parser.add_argument('-d', '--dir_path', type=str, default=(os.getcwd() + '\\data\\part1'),
                        help='Full path of dataset containing images for training the model')
    parser.add_argument('-t', '--test_images_path', type=str, default=(os.getcwd() + '\\data\\test_images'),
                        help='Full path of dataset containing test images')
    parser.add_argument('--csv_file_dir', type=str, default=(os.getcwd() + '\\data\\part2'),
                        help='CSV file directory having bbox for images')
    parser.add_argument('-m', '--pretrained_model', type=str.lower,
                        choices=['resnet18', 'resnet34', 'resnet50', 'densenet161', 'inceptionresnetv2'],
                        default='resnet18', help='Pretrained base model')
    parser.add_argument('-e', '--epochs', type=int, default=20,  help='No. of epochs')
    parser.add_argument('--optimizer', type=str.lower, choices=['sgd', 'adam'],
                        default='sgd', help='Optimizer function')
    parser.add_argument('-bs', '--batch_size', type=int, default=32, help='Batch size')
    parser.add_argument('-lr', '--learning_rate', type=float, default=0.0001, help='Learning rate')
    parser.add_argument('--show_plot', type=bool, choices=[True, False], default=True,
                        help='Display/Show corresponding plot')
    # parser.add_argument('-v', '--verbose', action="store_true",
    #                     help='Increase output verbosity')

    return parser.parse_args(args=[])


In [4]:
def main():
    args = parse_cmdline_arguments()
    
    # Check if the dataset directory exists and import the information about the players
    helpers.check_path_exists(args.dir_path, 'train images directory')
    df_player_info, teams_dic, players_dic = helpers.generate_player_info(args.dir_path, check_invalid_images_flag=False)

    # Split the data and apply the transformations (resize, normalize, etc)
    data = helpers.split_data(df_player_info, test_size=0.2)
    train_transform = transforms.Compose([tr.Resize(224, 224), tr.ToTensor(), tr.Normalization()])
    test_transform = transforms.Compose([tr.Resize(224, 224), tr.ToTensor(), tr.Normalization()])
    data['train'] = classification.Classification(player_info=data['train'], transform=train_transform)
    data['valid'] = classification.Classification(player_info=data['valid'], transform=train_transform)

    # Generate the dataloaders for training and validation set
    train_dataloader = DataLoader(dataset=data['train'], batch_size=args.batch_size, shuffle=True, num_workers=0)
    valid_dataloader = DataLoader(dataset=data['valid'], batch_size=args.batch_size, shuffle=False, num_workers=0)
    dataloaders = {phase:train_dataloader if phase == 'train' else valid_dataloader for phase in ['train', 'valid']}

    # Define the model and move to the device available
    # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    device = torch.device("cpu")
    model_CNN = model.Model(True, args.pretrained_model, len(teams_dic), len(players_dic), data, args)

    # Define the Optimizer and Loss function
    criterions, optimizer = model_CNN.compilation_parameters(model_CNN, args)
    

    # Perform the model training
    if args.perform == 'train' or args.perform == 'train_and_predict':
        model_CNN = model_CNN.train_model(model_CNN, dataloaders, criterions, optimizer, device, n_epochs=args.epochs,
        show_plot=args.show_plot)
        torch.save(model_CNN.state_dict(), 'model_CNN.ckpt')
    
    # Perform prediction on the images present in the specified directory
    if args.perform == 'predict' or args.perform == 'train_and_predict':
        model_CNN.load_state_dict(torch.load('model_CNN.ckpt'))
        predict = prediction.Prediction(model_CNN, test_transform, teams_dic, players_dic)            
        helpers.check_path_exists(args.test_images_path, 'test images directory')
        predict.predict_images_in_directory(args.test_images_path, check_invalid_images_flag=False)
        
    # Perform prediction and color the section of images as per the prediction outcome
    if args.perform == 'predict_and_color' or args.perform == 'train_and_predict':
        model_CNN.load_state_dict(torch.load('model_CNN.ckpt'))
        predict = prediction.Prediction(model_CNN, test_transform, teams_dic, players_dic)        
        predict.predict_and_color_section_of_images(args.csv_file_dir, show_plot=args.show_plot)


In [5]:
if __name__ == '__main__':
    main()

Epoch: 1 	Training Acc: 0.0223 	Validation Acc: 0.0303
time for epoch: 5.08 mins
Epoch: 2 	Training Acc: 0.0488 	Validation Acc: 0.0481
time for epoch: 4.9 mins
Epoch: 3 	Training Acc: 0.0595 	Validation Acc: 0.0597
time for epoch: 5.04 mins
Epoch: 4 	Training Acc: 0.0631 	Validation Acc: 0.0609
time for epoch: 4.98 mins
Epoch: 5 	Training Acc: 0.0621 	Validation Acc: 0.0672
time for epoch: 4.93 mins
Epoch: 6 	Training Acc: 0.0571 	Validation Acc: 0.0758
time for epoch: 5.45 mins
Epoch: 7 	Training Acc: 0.0519 	Validation Acc: 0.0906
time for epoch: 5.32 mins
Epoch: 8 	Training Acc: 0.0476 	Validation Acc: 0.0908
time for epoch: 5.12 mins
Epoch: 9 	Training Acc: 0.0448 	Validation Acc: 0.0981
time for epoch: 5.22 mins
Epoch: 10 	Training Acc: 0.0430 	Validation Acc: 0.1111
time for epoch: 4.97 mins


FileNotFoundError: No such file: 'd:\OneDrive\Desktop\player_and_team_recognition\data\part1_all\nottingham_forrest\person_28\41.jpg'